In [ ]:
import contextlib, json, os, pathlib

if (workdir := pathlib.Path('/home/jovyan/work')).exists():
    os.chdir(workdir)
elif pathlib.Path('.jupyterlab/notebooks/DUO').exists():
    os.chdir('.jupyterlab/notebooks')
elif pathlib.Path('.').absolute().name == 'DUO':
    os.chdir('..')
elif pathlib.Path('.').absolute().name == 'notebooks':
    pass
else:
    raise ValueError(os.getcwd())
print(os.getcwd())

%run ~/work/__init__edwh__new.ipynb
# !pip install pydal httpx[http2] trio typedict
!pip install geopy

import pydal
from pydal import DAL, Field
import httpx, trio
from tabulate import tabulate
import typedict as typedict
from geopy.distance import geodesic as distance 

In [ ]:
if 'duodb' not in locals():
    duodb = pydal.DAL('sqlite://duo-data.sqlite3', folder='./DUO', auto_import=True)
    DUODB = "sqlite:///DUO/db/duo-data.sqlite3"
print(len(duodb.tables),'tabellen in duodata')

In [ ]:
%%sql $PG 
select id, name, effdt, prio, effstatus, gid from organisation where gid like 'c5c83398%'

In [ ]:
%sql select prio, count(*) from organisation_effdted_now group by prio

In [ ]:
%%sql $DUODB
select * from edwh_onderwijslocaties limit 10 

In [ ]:
%%sql $PG
select count(distinct coc) as cocs, count(distinct gid) as gids, count(distinct brin) as brins from organisation

## Opschonen van ongebruikte organisaties

Organisaties die we nu niet gebruiken kunnen we het beste eerst verwijderen, dat scheelt veel met het matchmaken en errors, omdat we weten dat de scholen die overblijven, er ook echt toe doen.

Een organisatie is ongebruikt als:
 * er geen items aan hangen, dwz middels een tag koppeling in de `mv__item_tags` tabel
 * er geen (j)eddie wijzigingen in de org tabel zijn geweest, dwz de `prio <= 100` (want 0 == history en 100 == DUO)


In [ ]:
%%sql $PG
select org.name, count(mit.*) as items, max(org.prio) as prio
  from organisation_effdted_now org
  left outer join mv__item_tags mit on org.tag_gid::uuid = mit.tag_gid
group by org.name
having count(mit.*) = 0 and max(org.prio) <= 100
order by 3 desc
limit 10

gebruik bovenstaande query om alle organisaties te verwijderen die niet gebruikt worden

In [ ]:
%%sql $PG 
delete from organisation where gid in (
  select org.gid
    from organisation_effdted_now org
    left outer join mv__item_tags mit on org.tag_gid::uuid = mit.tag_gid
  group by org.gid
  having count(mit.*) = 0 and max(org.prio) <= 100
)



## Onderzoek van identifiers
Van de beschikbare identifiers zijn er best veel beschikbaar. Nu we de tabel hebben geschoond van niet relevante organisaties,
houden we een lijst over van waar we echt *wel* op moeten zoeken.

In [ ]:
scan = %sql select case when prio > 0 then 1 else 0 end as handmatig, name, coc, street, number, city, lonlat, brin, vestigingscode  from organisation_effdted_now
counted = [(row.count(None), row) for row in scan if row.count(None) > 0]
counted.sort(key=lambda row: row[0], reverse=True)
counted[0:5]

In [ ]:
import math 
from dataclasses import dataclass 
@dataclass 
class Point:
    gid:uuid.UUID
    latlon:tuple[int,int]
    
    def __post_init__(self):
        self.latlon = float(self.latlon[0]), float(self.latlon[1])
        
    def __hash__(self):
        return self.gid.int
    
    def near(self, b:"Point", level:int=10) -> bool:
        """Level == 7 roughly for half a km."""
        alat, alon = self.latlon
        blat, blon = b.latlon 
        difflat = int((alat-blat) * 1000) 
        difflon = int((alon-blon) * 1000)
        return abs(difflat) + abs(difflon)< level
            
        # avoid the sqrt because power is quicker 
        # using powers avoid the use for abs, because each negative float squared is positive anyway
        # dist = (difflat**2 + difflon**2)
        # result = dist < level**2
        # m = self.distance(b).m
        # if m < 1000:
        #     print(f'{difflat:6} {difflon:6} -> {dist:8} | {m:12}')
        # return result
   
    def distance(self, other):
        return distance(self.latlon, other.latlon)
    
lonlats = %sql select gid, lonlat from organisation_effdted_now 
lonlats = [Point(uuid.UUID(gid),tuple(lonlat.strip('()').split(',')[::-1])) for gid, lonlat in lonlats if lonlat]
lonlats[:10]

In [ ]:
distances = {}
from geopy.distance import geodesic as distance 
for source in tqdm(lonlats):
    distances[source] = {other:source.distance(other) for other in lonlats if other != source and source.near(other)}
    

In [ ]:
# Only keep keys when optional near locations are available. 
distances = {k:v for k,v in distances.items() if v}
for key in list(distances.keys())[:5]:
    print('##',key.gid)
    for other, distance in distances[key].items():
        print(other.gid, distance.m)

### Controle van dubbele organisaties:
er zijn 2 scholen in de data die nu dubbel geregistreerd lijken te zijn, maar waaraan ook artikelen gekoppeld zitten, of ze zijn door een %eddie bewerkt.

In [ ]:
%%sql
select concat(city,'-',postalcode,'-',street,'-', number) as adres, count(*), json_agg(name) as namen, json_agg(substr(gid, 0,9)) as short_gids 
from organisation_effdted_now 
group by adres 
having count(*) > 1


## Zoek naar brinlose organisaties
Uit de subset van bestaande scholen, is `brin` de beste koppeling, zodat we vanuit daar kunnen koppelen met de `onderwijslocatiecode` om gegevens echt goed koppelbaar te maken. 
Daarom moeten we eerst de scholen allemaal voorzien van de brincode om daarna verder te koppelen.  
:warn: Ook moeten we natuurlijk controleren dat de Brin wel klopt met onze gegevens! 

Ook leuk, we halen nog even de oude brin koppeling erbij. Dat kan **misschien** iets helpen: helaas, alles wat we nu vinden aan organisaties *lijken* later toegevoegd, want deze gids komen niet voor in de `all_with_gid.db` 

In [ ]:
originele_import = %sql sqlite:///DUO/all_with_gid.db select * from scholen 
originele_import = list(originele_import.dicts())
originele_organisaties_gid_map = {i['org_gid']:i for i in originele_import}

In [ ]:
%%sql $PG
select brin, vestigingscode, count(*) from organisation_effdted_now
group by brin, vestigingscode
having count(*) > 1
order by count(*) desc

Van deze organisaties moeten willen we de rest van de gegevens weten:

In [ ]:
bij_te_werken_organisaties = %sql $PG select gid, substr(gid,0,9) as short_gid, tag_gid, substr(tag_gid,0,9)as short_tag_gid, name, coc, street, number, city, lonlat, brin, vestigingscode, postalcode, is_active from organisation_effdted_now where brin is null or vestigingscode is null 
bij_te_werken_organisaties

In [ ]:
from pprint import pprint
from collections import Counter 
gevonden = {}
organisaties_effdted_now = %sql $PG select * from organisation_effdted_now
organisaties_gid_map = {org.gid:org for org in organisaties_effdted_now}

for ori in originele_import: 
    zoek_gid = ori['org_gid'] # voeg quotes toe, want die zijn hieronder niet toe te voegen 
    gevonden[zoek_gid] = zoek_gid in organisaties_gid_map
Counter(gevonden.values())


In [ ]:
for bij_te_werken_org in bij_te_werken_organisaties:
    if bij_te_werken_org.gid in originele_organisaties_gid_map:
        print('bij te werken komt voor in oude import')
        oude_org = originele_organisaties_gid_map[bij_te_werken_org.gid]
        print(oude_org['BRIN NUMMER'], oude_org['VESTIGINGSNUMMER'], oude_org['POSTCODE'], oude_org['HUISNUMMER-TOEVOEGING'])
        #pprint(oude_org)
    else: 
        print('bij te werken komt niet voor in oude import')
## HIER WAS IK GEBLEVEN

In [ ]:
%%sql $PG
select * from organisation_effdted_now limit 5

## Views maken om DUO data te combineren:

In [ ]:
%%sql $DUODB
select * from aangeboden_opleiding limit 10

In [ ]:
%%sql $DUODB
select * from edwh_onderwijslocaties limit 10

# Taken:
 []: Controle op brin en vestigingscode als vinkje in organisatie overzicht (aanwezig en combi is uniek)
 []: Alarm wanneer combinatie plaats-straat-nummer niet uniek zijn (zou kunnen gebeuren, maar is zeer onwaarschijnlijk)
 []: